**Table of contents**<a id='toc0_'></a>    
- [image based visual servo (IBVS)](#toc1_)    
- [Starting pybullet](#toc2_)    
- [Initial Setup for pybullet](#toc3_)    
- [Generating the Robot Arm](#toc4_)    
- [Generating the Box with AR Marker Attached](#toc5_)    
- [Changing the Light Source Position](#toc6_)    
- [Defining Functions to be Used](#toc7_)    
- [Setting Camera Parameters](#toc8_)    
- [Setting Parameters for AR Markers](#toc9_)    
- [Setting Parameters for the Robot Arm](#toc10_)    
- [Executing Visual Servoing](#toc11_)    
- [Bonus: Drawing Feature Point Positions](#toc12_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[image based visual servo (IBVS)](#toc0_)

In this notebook, we will introduce a method to perform image-based visual servoing using a 6-axis robot arm.

Visual servoing reference: https://github.com/RiddhimanRaut/Ur5_Visual_Servoing/blob/master/ur5_control_nodes/src/vs_ur5.py

(For a manual summarizing the functions available in pybullet, refer to [this link](https://github.com/bulletphysics/bullet3/blob/master/docs/pybullet_quickstartguide.pdf).)

# <a id='toc2_'></a>[Starting pybullet](#toc0_)

In [1]:
import pybullet
import pybullet_data
physics_client = pybullet.connect(pybullet.GUI) 

pybullet build time: Nov 28 2023 23:45:17


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa
GL_RENDERER=llvmpipe (LLVM 15.0.7, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Mesa
Renderer = llvmpipe (LLVM 15.0.7, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


# <a id='toc3_'></a>[Initial Setup for pybullet](#toc0_)

In [2]:
pybullet.resetSimulation() # Reset the simulation space
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath()) # Add paths to necessary data for pybullet
# pybullet.setGravity(0.0, 0.0, -9.8) # Set gravity as on Earth (for simplicity, gravity is not considered this time)
time_step = 1./240.
pybullet.setTimeStep(time_step) # Set the time elapsed per step

# Load the floor
plane_id = pybullet.loadURDF("plane.urdf")

# Set the camera position and other parameters in GUI mode
camera_distance = 2.0
camera_yaw = 0.0 # deg
camera_pitch = -20 # deg
camera_target_position = [0.0, 0.0, 0.0]
pybullet.resetDebugVisualizerCamera(camera_distance, camera_yaw, camera_pitch, camera_target_position)

ven = Mesa
ven = Mesa


# <a id='toc4_'></a>[Generating the Robot Arm](#toc0_)

In [3]:
# Load the robot
arm_start_pos = [0, 0, 0]  # Set the initial position (x, y, z)
arm_start_orientation = pybullet.getQuaternionFromEuler([0, 0, 0])  # Set the initial orientation (roll, pitch, yaw)
arm_id = pybullet.loadURDF("../urdf/simple6d_arm_with_force_sensor.urdf", arm_start_pos, arm_start_orientation, useFixedBase=True) # Fix the root link with useFixedBase=True to prevent the robot from falling

# Set the camera position and other parameters in GUI mode
camera_distance = 1.5
camera_yaw = 50.0 # deg
camera_pitch = -40 # deg
camera_target_position = [0.0, 0.0, 1.0]
pybullet.resetDebugVisualizerCamera(camera_distance, camera_yaw, camera_pitch, camera_target_position)

b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: target_position_vertual_link


# <a id='toc5_'></a>[Generating the Box with AR Marker Attached](#toc0_)

In [4]:
# Generate a box with an AR marker drawn on one side
box_bid = pybullet.loadURDF("../urdf/ar_marker_box.urdf", [0, 0, 0.5], useFixedBase=True)

# Set the texture (specify the same one as in the urdf file)
texture_id = pybullet.loadTexture("../texture/ar_marker_box.png")
pybullet.changeVisualShape(box_bid, -1, textureUniqueId=texture_id)

# Set sliders to adjust the position of the box with the AR marker attached
pybullet.addUserDebugParameter("obj_x", -4, 4, 0.0)
pybullet.addUserDebugParameter("obj_y", -2, 0, -1.5)
pybullet.addUserDebugParameter("obj_z", -4, 8, 1.3)
pybullet.addUserDebugParameter("obj_roll", -3.14, 3.14, 0.0)
pybullet.addUserDebugParameter("obj_pitch", -3.14, 3.14, 0)
pybullet.addUserDebugParameter("obj_yaw", -3.14, 3.14, 0)

5

# <a id='toc6_'></a>[Changing the Light Source Position](#toc0_)

In [5]:
# Change the light source position because the AR marker is hard to recognize with the default light source position
pybullet.configureDebugVisualizer(lightPosition=[0, 0, 10])

# <a id='toc7_'></a>[Defining Functions to be Used](#toc0_)

In [ ]:
import numpy as np
import cv2

def Rx(theta):
    """
    Calculate the rotation matrix around the x-axis

    Parameters
    ----------
    theta : float
        Rotation angle [rad]

    Returns
    -------
    numpy.ndarray
        Rotation matrix around the x-axis
    """
    return np.array([[1, 0, 0],
                     [0, np.cos(theta), -np.sin(theta)],
                     [0, np.sin(theta), np.cos(theta)]])

def Ry(theta):
    """
    Calculate the rotation matrix around the y-axis

    Parameters
    ----------
    theta : float
        Rotation angle [rad]

    Returns
    -------
    numpy.ndarray
        Rotation matrix around the y-axis
    """
    return np.array([[np.cos(theta), 0, np.sin(theta)],
                     [0, 1, 0],
                     [-np.sin(theta), 0, np.cos(theta)]])

def Rz(theta):
    """
    Calculate the rotation matrix around the z-axis

    Parameters
    ----------
    theta : float
        Rotation angle [rad]

    Returns
    -------
    numpy.ndarray
        Rotation matrix around the z-axis
    """
    return np.array([[np.cos(theta), -np.sin(theta), 0],
                     [np.sin(theta), np.cos(theta), 0],
                     [0, 0, 1]])


def Hx(theta):
    """
    Calculate the homogeneous transformation matrix around the x-axis

    Parameters
    ----------
    theta : float
        Rotation angle [rad]

    Returns
    -------
    numpy.ndarray
        Homogeneous transformation matrix around the x-axis
    """
    return np.array([[1, 0, 0, 0],
                     [0, np.cos(theta), -np.sin(theta), 0],
                     [0, np.sin(theta), np.cos(theta), 0],
                     [0, 0, 0, 1]])

def Hy(theta):
    """
    Calculate the homogeneous transformation matrix around the y-axis

    Parameters
    ----------
    theta : float
        Rotation angle [rad]

    Returns
    -------
    numpy.ndarray
        Homogeneous transformation matrix around the y-axis
    """
    return np.array([[np.cos(theta), 0, np.sin(theta), 0],
                     [0, 1, 0, 0],
                     [-np.sin(theta), 0, np.cos(theta), 0],
                     [0, 0, 0, 1]])

def Hz(theta):
    """
    Calculate the homogeneous transformation matrix around the z-axis

    Parameters
    ----------
    theta : float
        Rotation angle [rad]

    Returns
    -------
    numpy.ndarray
        Homogeneous transformation matrix around the z-axis
    """
    return np.array([[np.cos(theta), -np.sin(theta), 0, 0],
                     [np.sin(theta), np.cos(theta), 0, 0],
                     [0, 0, 1, 0],
                     [0, 0, 0, 1]])

def Hp(x, y, z):
    """
    Calculate the homogeneous transformation matrix for translation

    Parameters
    ----------
    x : float
        Translation in the x direction
    y : float
        Translation in the y direction
    z : float
        Translation in the z direction

    Returns
    -------
    numpy.ndarray
        Homogeneous transformation matrix for translation
    """
    return np.array([[1, 0, 0, x],
                     [0, 1, 0, y],
                     [0, 0, 1, z],
                     [0, 0, 0, 1]])
                     
def make_6d_jacobian_matrix(R, o1, o2, o3, o4, o5, o6, oc):
    """
    Calculate the basic Jacobian matrix of a 6-axis robot arm

    Parameters
    ----------
    R : numpy.ndarray
        Rotation matrices between coordinate systems of the robot arm
    o1 : numpy.ndarray
        Origin of link1
    o2 : numpy.ndarray
        Origin of link2
    o3 : numpy.ndarray
        Origin of link3
    o4 : numpy.ndarray
        Origin of link4
    o5 : numpy.ndarray
        Origin of link5
    o6 : numpy.ndarray
        Origin of link6
    oc : numpy.ndarray
        Origin of the camera
   
        
    Returns
    -------
    Jv : numpy.ndarray
         Basic Jacobian matrix
    """

    
    R12 = R[0]
    R23 = R[1]
    R34 = R[2]
    R45 = R[3]
    R56 = R[4]
    R6C = R[5]
    ex = np.array([1, 0, 0])
    ey = np.array([0, 1, 0])
    ez = np.array([0, 0, 1])

    # Calculate each element of the Jacobian matrix
    a1 = R12 @ ez
    a2 = R12 @ R23 @ ey
    a3 = R12 @ R23 @ R34 @ ey
    a4 = R12 @ R23 @ R34 @ R45 @ ez
    a5 = R12 @ R23 @ R34 @ R45 @ R56 @ ey
    a6 = R12 @ R23 @ R34 @ R45 @ R56 @ R6C @ ez

    # Ji = [[ai x (oc - oi)], 
    #       [ai]]
    j1 = np.concatenate((np.cross(a1, oc-o1).reshape(3, 1), a1.reshape(3, 1)), axis=0)
    j2 = np.concatenate((np.cross(a2, oc-o2).reshape(3, 1), a2.reshape(3, 1)), axis=0)
    j3 = np.concatenate((np.cross(a3, oc-o3).reshape(3, 1), a3.reshape(3, 1)), axis=0)
    j4 = np.concatenate((np.cross(a4, oc-o4).reshape(3, 1), a4.reshape(3, 1)), axis=0)
    j5 = np.concatenate((np.cross(a5, oc-o5).reshape(3, 1), a5.reshape(3, 1)), axis=0)
    j6 = np.concatenate((np.cross(a6, oc-o6).reshape(3, 1), a6.reshape(3, 1)), axis=0)

    Jv = np.concatenate((j1, j2, j3, j4, j5, j6), axis=1)
    return Jv

def make_4features_image_jacobian_matrix(current_feature_points, camera_to_obj, f):
    """
    Calculate the image Jacobian matrix for 4 feature points

    Parameters
    ----------
    current_feature_points : numpy.ndarray
        Current feature point coordinates
    camera_to_objs : float
        Distance from the camera to the object
    f : float
        Focal length of the camera
    
    Returns
    -------
    Ji : numpy.ndarray
        Image Jacobian matrix
    """
    # Image Jacobian matrix for 4 feature points
    u1 = current_feature_points[0]
    v1 = current_feature_points[1]
    u2 = current_feature_points[2]
    v2 = current_feature_points[3]
    u3 = current_feature_points[4]
    v3 = current_feature_points[5]
    u4 = current_feature_points[6]
    v4 = current_feature_points[7]

    Ji = np.array([[-f*(1/camera_to_obj), 0, u1*(1/camera_to_obj), u1*v1/f, -(f+(1/f)*u1**2), v1],
                   [0, -f*(1/camera_to_obj), v1*(1/camera_to_obj), f+(1/f)*v1**2, -(1/f)*u1*v1, -u1],
                   [-f*(1/camera_to_obj), 0, u2*(1/camera_to_obj), u2*v2/f, -(f+(1/f)*u2**2), v2],
                   [0, -f*(1/camera_to_obj), v2*(1/camera_to_obj), f+(1/f)*v2**2, -(1/f)*u2*v2, -u2],
                   [-f*(1/camera_to_obj), 0, u3*(1/camera_to_obj), u3*v3/f, -(f+(1/f)*u3**2), v3],
                   [0, -f*(1/camera_to_obj), v3*(1/camera_to_obj), f+(1/f)*v3**2, -(1/f)*u3*v3, -u3],
                   [-f*(1/camera_to_obj), 0, u4*(1/camera_to_obj), u4*v4/f, -(f+(1/f)*u4**2), v4],
                   [0, -f*(1/camera_to_obj), v4*(1/camera_to_obj), f+(1/f)*v4**2, -(1/f)*u4*v4, -u4]])
    return Ji

def detect_ar_marker_corner_pos_and_depth(marker_size, aruco_dict, parameters, rgb_img, camera_matrix, dist_coeffs):
    """
    Detect the positions of the four corners of the AR marker and the distance to the camera

    Parameters
    ----------
    marker_size : float
        Length of one side of the AR marker (meters)
    aruco_dict : cv2.aruco.Dictionary
        Dictionary of AR markers
    parameters : cv2.aruco.DetectorParameters
        Parameters for AR marker detection
    rgb_img : numpy.ndarray
        Camera image (RGB)
    camera_matrix : numpy.ndarray
        Camera intrinsic parameter matrix
    dist_coeffs : numpy.ndarray
        Distortion coefficients

    Returns
    -------
    corner_pos : numpy.ndarray
        Positions of the four corners of the AR marker (x1, y1, x2, y2, x3, y3, x4, y4) [pixel positions]
    depth : float
        Distance from the camera to the AR marker [m]
    """

    # Convert the camera image to grayscale
    rgb_img = cv2.cvtColor(rgb_img, cv2.COLOR_RGB2BGR)

    # Detect AR markers
    corners, ids, _ = cv2.aruco.detectMarkers(rgb_img, aruco_dict, parameters=parameters)

    # Return None if no markers are detected
    if ids is None or len(ids) == 0:
        return None, None, None

    # Get the positions of the four corners of the first detected marker
    marker_corners = corners[0][0]

    # Get the positions and corresponding depths of each corner
    corner_pos = np.zeros(8)
    for i in range(4):
        x, y = int(marker_corners[i][0]), int(marker_corners[i][1])
        corner_pos[i*2] = x
        corner_pos[i*2+1] = y

    # Calculate the distance from the camera to the AR marker using solvePnP
    # 3D coordinates
    corner_points_3d = np.array([[-marker_size/2, -marker_size/2, 0],
                                 [marker_size/2, -marker_size/2, 0],
                                 [marker_size/2, marker_size/2, 0],
                                 [-marker_size/2, marker_size/2, 0]], dtype=np.float32)
    # 2D coordinates
    corner_points_2d = np.array([[corner_pos[0], corner_pos[1]],
                                 [corner_pos[2], corner_pos[3]],
                                 [corner_pos[4], corner_pos[5]],
                                 [corner_pos[6], corner_pos[7]]], dtype=np.float32)

    # Calculate the distance from the camera to the AR marker using solvePnP
    _, _, translation_vector = cv2.solvePnP(corner_points_3d, corner_points_2d, camera_matrix, dist_coeffs)

    # Calculate the distance from the camera to the AR marker
    depth = translation_vector[2][0]

    return corner_pos, depth

# <a id='toc8_'></a>[Setting Camera Parameters](#toc0_)

In [7]:
# Camera settings
fov = 60
image_width = 224
image_height = 224
center_x = image_width / 2
center_y = image_height / 2
aspect = image_width / image_height
near = 0.05
far = 10
projection_matrix = pybullet.computeProjectionMatrixFOV(fov, aspect, near, far)

# Calculate the focal length
fov_rad = np.deg2rad(fov)
f = image_height / (2 * np.tan(fov_rad / 2))

# Camera intrinsic parameters
camera_matrix = np.array([[f, 0, image_width/2],
                         [0, f, image_height/2],
                         [0, 0, 1]], dtype=np.float32)

# Distortion coefficients (assuming no distortion here)
dist_coeffs = np.array([0.0, 0.0, 0.0, 0.0, 0.0], dtype=np.float32)

# <a id='toc9_'></a>[Setting Parameters for AR Markers](#toc0_)

In [8]:
marker_size = 0.1 # Length of one side of the AR marker (meters)

# Define the dictionary for the AR marker to be detected
aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_50)
parameters = cv2.aruco.DetectorParameters_create()

# <a id='toc10_'></a>[Setting Parameters for the Robot Arm](#toc0_)

In [9]:
# Length of each link
ARM_ORIGIN_X_WORLD = 0.0 # x-coordinate of the arm's origin in the world coordinate system
ARM_ORIGIN_Y_WORLD = 0.0 # y-coordinate of the arm's origin in the world coordinate system
ARM_ORIGIN_Z_WORLD = 0.8 # z-coordinate of the arm's origin in the world coordinate system (length of base_link in "simple6d_arm_with_gripper.urdf")
LINK1_LENGTH = 0.3 # Length of link1 (z-direction length of link1 in "simple6d_arm_with_gripper.urdf")
LINK2_LENGTH = 0.5 # Length of link2 (z-direction length of link2 in "simple6d_arm_with_gripper.urdf")
LINK3_LENGTH = 0.5 # Length of link3 (z-direction length of link3 in "simple6d_arm_with_gripper.urdf")
LINK4_LENGTH = 0.1 # Length of link4 (z-direction length of link4 in "simple6d_arm_with_gripper.urdf")
LINK5_LENGTH = 0.1 # Length of link5 (z-direction length of link5 in "simple6d_arm_with_gripper.urdf")
LINK6_LENGTH = 0.15 # Length of link6 (z-direction length of link6 in "simple6d_arm_with_gripper.urdf")
CAMERA_Z_SIZE = 0.01 # Size of the camera in the z-direction
CAMERA_X_OFFSET = 0.08 # x-direction offset from the origin of the link6 coordinate system to the origin of the camera coordinate system

# Changing these values will change the results #############
# Define the initial angles of each link
link1_angle_deg = -90
link2_angle_deg = 55
link3_angle_deg = 35
link4_angle_deg = 0
link5_angle_deg = 0
link6_angle_deg = 0
feature_param = 10

# Target positions of the 4 features on the image
goal1_u = center_x - 50
goal1_v = center_y - 50
goal2_u = center_x + 50
goal2_v = center_y - 50
goal3_u = center_x + 50
goal3_v = center_y + 50
goal4_u = center_x - 50
goal4_v = center_y + 50
##############################################

# Convert degrees to radians
link1_angle_rad  = np.deg2rad(link1_angle_deg)
link2_angle_rad  = np.deg2rad(link2_angle_deg)
link3_angle_rad  = np.deg2rad(link3_angle_deg)
link4_angle_rad  = np.deg2rad(link4_angle_deg)
link5_angle_rad  = np.deg2rad(link5_angle_deg)
link6_angle_rad  = np.deg2rad(link6_angle_deg)

# List the target positions of the 4 features
goal_feature_points = np.array([goal1_u, goal1_v, goal2_u, goal2_v, goal3_u, goal3_v, goal4_u, goal4_v])

# Indices of each joint of the robot
LINK1_JOINT_IDX = 0
LINK2_JOINT_IDX = 1
LINK3_JOINT_IDX = 2
LINK4_JOINT_IDX = 3
LINK5_JOINT_IDX = 4
LINK6_JOINT_IDX = 5
CAMERA_IDX = 6
CAMERA_TARGET_IDX = 7

# Initialize the angles of each joint of the robot
pybullet.resetJointState(arm_id, LINK1_JOINT_IDX, link1_angle_rad )
pybullet.resetJointState(arm_id, LINK2_JOINT_IDX, link2_angle_rad )
pybullet.resetJointState(arm_id, LINK3_JOINT_IDX, link3_angle_rad )
pybullet.resetJointState(arm_id, LINK4_JOINT_IDX, link4_angle_rad )
pybullet.resetJointState(arm_id, LINK5_JOINT_IDX, link5_angle_rad )
pybullet.resetJointState(arm_id, LINK6_JOINT_IDX, link6_angle_rad )

# <a id='toc11_'></a>[Executing Visual Servoing](#toc0_)

You can change the position and orientation of the box with the AR marker attached by adjusting the values of the sliders on the right.

Note that with the default robot posture and box position, changes in the "x, roll, yaw" directions may not work properly unless adjusted carefully.  
Therefore, it is recommended to start with changes in the "y, z, pitch" directions.

In [10]:
import threading
import time

# Global variables (variables used in the bonus chapter are defined as global variables)
current_feature_points = None # Current position of feature points
rgb_img = None # RGB image obtained from the camera

def VisualServo():
    global current_feature_points
    global rgb_img

    while True:
        # Set the "box with AR marker attached" to the position and orientation set by the slider
        obj_x = pybullet.readUserDebugParameter(0)
        obj_y = pybullet.readUserDebugParameter(1)
        obj_z = pybullet.readUserDebugParameter(2)
        obj_roll = pybullet.readUserDebugParameter(3)
        obj_pitch = pybullet.readUserDebugParameter(4)
        obj_yaw = pybullet.readUserDebugParameter(5)
        pybullet.resetBasePositionAndOrientation(box_bid, [obj_x, obj_y, obj_z], pybullet.getQuaternionFromEuler([obj_roll, obj_pitch, obj_yaw]))

        # Get joint angles
        q1 = pybullet.getJointState(arm_id, LINK1_JOINT_IDX)[0]
        q2 = pybullet.getJointState(arm_id, LINK2_JOINT_IDX)[0]
        q3 = pybullet.getJointState(arm_id, LINK3_JOINT_IDX)[0]
        q4 = pybullet.getJointState(arm_id, LINK4_JOINT_IDX)[0]
        q5 = pybullet.getJointState(arm_id, LINK5_JOINT_IDX)[0]
        q6 = pybullet.getJointState(arm_id, LINK6_JOINT_IDX)[0]

        # Set the camera's up vector according to the pose of the camera at the robot arm's end-effector
        camera_up_vector = np.array([0, -1, 0]) # Default camera up vector
        RW1 = Rz(theta=0)@Ry(theta=0)@Rx(theta=0) # World coordinate system -> link1 coordinate system
        R12 = Rz(theta=q1) # link1 coordinate system -> link2 coordinate system
        R23 = Ry(theta=q2) # link2 coordinate system -> link3 coordinate system
        R34 = Ry(theta=q3) # link3 coordinate system -> link4 coordinate system
        R45 = Rz(theta=q4) # link4 coordinate system -> link5 coordinate system
        R56 = Ry(theta=q5) # link5 coordinate system -> link6 coordinate system
        R6C = Rz(theta=q6-np.pi/2) # link6 coordinate system -> camera coordinate system (rotate the camera by -90 degrees to get the correct image orientation)
        RWC = RW1@R12@R23@R34@R45@R56@R6C # World coordinate system -> camera coordinate system
        rotate_camera_up_vector = RWC@camera_up_vector # Camera up vector in the camera coordinate system

        # Get camera image
        camera_link_pose = pybullet.getLinkState(arm_id, CAMERA_IDX)[0] # Position of the camera link at the end-effector
        cameraTargetLinkPose = pybullet.getLinkState(arm_id, CAMERA_TARGET_IDX)[0] # Position of a virtual link set slightly ahead of the camera link
        view_matrix = pybullet.computeViewMatrix(cameraEyePosition=[camera_link_pose[0], camera_link_pose[1], camera_link_pose[2]],cameraTargetPosition=[cameraTargetLinkPose[0], cameraTargetLinkPose[1], cameraTargetLinkPose[2]],cameraUpVector=rotate_camera_up_vector)
        _, _, rgb_img, _, _ = pybullet.getCameraImage(
            width=image_width,
            height=image_height,
            viewMatrix=view_matrix,
            projectionMatrix=projection_matrix,
            renderer=pybullet.ER_BULLET_HARDWARE_OPENGL
        )

        # Get the position of the four feature points in the image coordinate system (pixel position) and the distance from the camera to the object [m]
        current_feature_points, z = detect_ar_marker_corner_pos_and_depth(marker_size, aruco_dict, parameters, rgb_img, camera_matrix, dist_coeffs)

        # If feature points cannot be detected, proceed to the next loop
        if current_feature_points is None:
            continue

        # Image Jacobian matrix
        Ji = make_4features_image_jacobian_matrix(current_feature_points, z, f)
        Ji_inv = np.linalg.pinv(Ji) # Inverse of the image Jacobian matrix

        # Calculate the position of each link's coordinate system from the world coordinate system (used for calculating the basic Jacobian matrix)
        TW1 = Hz(theta=0)@Hy(theta=0)@Hx(theta=0)@Hp(x=ARM_ORIGIN_X_WORLD, y=ARM_ORIGIN_Y_WORLD, z=ARM_ORIGIN_Z_WORLD) # World coordinate system -> link1 coordinate system
        T12 = Hz(theta=q1)@Hp(x=0, y=0, z=LINK1_LENGTH) # link1 coordinate system -> link2 coordinate system
        T23 = Hy(theta=q2)@Hp(x=0, y=0, z=LINK2_LENGTH) # link2 coordinate system -> link3 coordinate system
        T34 = Hy(theta=q3)@Hp(x=0, y=0, z=LINK3_LENGTH) # link3 coordinate system -> link4 coordinate system
        T45 = Hz(theta=q4)@Hp(x=0, y=0, z=LINK4_LENGTH) # link4 coordinate system -> link5 coordinate system
        T56 = Hy(theta=q5)@Hp(x=0, y=0, z=LINK5_LENGTH) # link5 coordinate system -> link6 coordinate system
        T6C = Hz(theta=q6-np.pi/2)@Hp(x=CAMERA_X_OFFSET, y=0.0, z=LINK6_LENGTH-CAMERA_Z_SIZE/2) # link6 coordinate system -> camera coordinate system
        origin_pos = np.array([0, 0, 0, 1]) # Origin position
        origin_link1_world = TW1@origin_pos                         # Origin position of link1 coordinate system from the world coordinate system
        origin_link2_world = TW1@T12@origin_pos                     # Origin position of link2 coordinate system from the world coordinate system
        origin_link3_world = TW1@T12@T23@origin_pos                 # Origin position of link3 coordinate system from the world coordinate system
        origin_link4_world = TW1@T12@T23@T34@origin_pos             # Origin position of link4 coordinate system from the world coordinate system
        origin_link5_world = TW1@T12@T23@T34@T45@origin_pos         # Origin position of link5 coordinate system from the world coordinate system
        origin_link6_world = TW1@T12@T23@T34@T45@T56@origin_pos     # Origin position of link6 coordinate system from the world coordinate system
        origin_linkC_world = TW1@T12@T23@T34@T45@T56@T6C@origin_pos # Origin position of camera coordinate system from the world coordinate system

        # From here, the main process of visual servoing ##########################################################
        # Vector from "current feature point position" to "target feature point position"
        feature_current_to_goal = goal_feature_points - current_feature_points

        # Small change amount towards the direction of "current feature point position" to "target feature point position"
        delta_feature_points = feature_param * feature_current_to_goal

        # Using the inverse of the image Jacobian matrix Ji_inv, calculate the small change amount of the camera (Δx, Δy, Δz, Δroll, Δpitch, Δyaw) camera_delta_p from the small change amount of the feature points delta_feature_points
        # (How to move the camera so that the feature points move to the target position)
        delta_p_camera = Ji_inv @ delta_feature_points

        # Diagonal matrix with R on the diagonal
        size = 6
        rotate_diagonal_matrix = np.zeros((size, size))
        for i in range(0, size, 3):
            rotate_diagonal_matrix[i:i+3, i:i+3] = RWC

        # Using the rotation matrix rotate_diagonal_matrix, convert the small change amount of the camera delta_p_camera based on the camera coordinate system to the small change amount of the camera delta_p_world based on the world coordinate system
        delta_p_world = rotate_diagonal_matrix @ delta_p_camera

        # Basic Jacobian matrix
        Jv = make_6d_jacobian_matrix(R=[R12, R23, R34, R45, R56, R6C],
                                     o1=origin_link1_world[0:3], 
                                     o2=origin_link2_world[0:3], 
                                     o3=origin_link3_world[0:3], 
                                     o4=origin_link4_world[0:3], 
                                     o5=origin_link5_world[0:3], 
                                     o6=origin_link6_world[0:3], 
                                     oc=origin_linkC_world[0:3])
        Jv_inv = np.linalg.pinv(Jv) # Inverse of the basic Jacobian matrix

        # Using the inverse of the basic Jacobian matrix Jv_inv, calculate the joint angular velocities delta_q from the small change amount of the camera delta_p_world based on the world coordinate system
        # (How to move each joint of the robot arm so that the camera moves to the target position and orientation)
        delta_q = Jv_inv @ delta_p_world

        # Set the calculated angular velocities to each joint
        pybullet.setJointMotorControl2(arm_id, LINK1_JOINT_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=delta_q[0])
        pybullet.setJointMotorControl2(arm_id, LINK2_JOINT_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=delta_q[1])
        pybullet.setJointMotorControl2(arm_id, LINK3_JOINT_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=delta_q[2])
        pybullet.setJointMotorControl2(arm_id, LINK4_JOINT_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=delta_q[3])
        pybullet.setJointMotorControl2(arm_id, LINK5_JOINT_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=delta_q[4])
        pybullet.setJointMotorControl2(arm_id, LINK6_JOINT_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=delta_q[5])

        # Advance by one time step
        pybullet.stepSimulation()
        time.sleep(time_step)

# Start the thread
thread = threading.Thread(target=VisualServo)
thread.start()

# <a id='toc12_'></a>[Bonus: Drawing Feature Point Positions](#toc0_)

By running the code below, the "goal feature point positions" and "current feature point positions" will be drawn.

You can specifically check how the feature points are moving.

In [11]:
import matplotlib.pyplot as plt
%matplotlib qt

plt.ion()  # Enable interactive mode
fig, ax = plt.subplots()
goal_plot, = ax.plot([], [], 'ro', label="Goal Points")
current_plot, = ax.plot([], [], 'bo', label="Current Points")
ax.set_xlim(0, image_width)
ax.set_ylim(0, image_height)
ax.invert_yaxis()  # Invert y-axis to match the image coordinate system

plt.legend()
plt.show()

# Main loop to draw the image obtained from the camera
initDraw = True
while True:
    # Draw rgb_img and plot goal_feature_points and current_feature_points
    try:
        if goal_feature_points is not None and current_feature_points is not None:
            if initDraw:
                img_plot = ax.imshow(rgb_img)
                initDraw = False
            else:
                img_plot.set_data(rgb_img)
            goal_plot.set_xdata([goal1_u, goal2_u, goal3_u, goal4_u])
            goal_plot.set_ydata([goal1_v, goal2_v, goal3_v, goal4_v])
            current_plot.set_xdata([current_feature_points[0], current_feature_points[2], current_feature_points[4], current_feature_points[6]])
            current_plot.set_ydata([current_feature_points[1], current_feature_points[3], current_feature_points[5], current_feature_points[7]])
            plt.pause(0.001)
    except:
        pass